In [2]:
# mode-n matricization of a tensor
import numpy as np

def tenmat(A,n):
    # A: the input tensor
    # n: the index
    N = A.ndim
    sizeA = np.array(A.shape)
    In = sizeA[n]
    Jn = int(A.size / In)
    
    perm = np.array(n)
    perm = np.append(perm,np.arange(n))
    perm = np.append(perm,np.arange(n+1,N))
    B = np.transpose(A,perm)
    A_mat = np.reshape(B,[In,Jn])
    
    return A_mat

In [3]:
# used to calculate a tensor times a matrix
import numpy as np

def ttm(A,U,n):
    # A: the input tensor
    # U: the input matrix 
    # n: the index (corresponds to the column of U)
    N = A.ndim
    sizeA = np.array(A.shape)
    sizeU = np.array(U.shape)
    
    sizeB = np.delete(sizeA,n)
    sizeB = np.insert(sizeB,0,sizeU[0])
    A_mat = tenmat(A,n)
    B = np.matmul(U,A_mat)
    B = np.reshape(B,sizeB)
    perm = np.arange(N)
    perm = np.delete(perm,0)
    perm = np.insert(perm,n,0)
    B = np.transpose(B,perm)
    
    return B

In [4]:
# used to calculate a tensor times a series of matrices
import numpy as np

def ttmc(A,U_list,op_index,transpose=False):
    # A: the input tensor
    # U_list: the input vectors (contracting the column)
    # op_index: the indices that need to be contracted (in tensor)
    
    B = np.array(A,copy=True)
    for i in range(len(U_list)):
        if transpose:
            B = ttm(B,U_list[i].T,op_index[i])
        else:
            B = ttm(B,U_list[i],op_index[i])

    return B

In [5]:
# used to calculate the rank-r approximation of a matrix by alternating least squares method with fixed error tolerance
import numpy as np

def als2_1(A,r,init="random",tol=1.0e-4,maxiter=500):
    # A: the input matrix
    # r: truncation
    # init: initial factors
    # tol: stop criterion
    # maxiter: the maximum number of iterations
    
    normA = np.linalg.norm(A)
    sizeA = A.shape
    num_iter = 0
    
    # initial factor
    if isinstance(init,np.ndarray):
        U_list = []
        U_list.append(init)
        U_list.append(np.random.random([sizeA[1],r]))
    elif init == "random":
        U_list = []
        U_list.append(np.random.random([sizeA[0],r]))
        U_list.append(np.random.random([sizeA[1],r]))
        
    B = np.matmul(U_list[0],U_list[1].T)
    res_old = np.linalg.norm(A - B)/normA
    res_change = res_old
    
#     print("\033[1;35;46mAlternating least squares method for matrix low-rank approximation:\n\033[0m")
    while (res_change > tol and num_iter < maxiter):
        # update the right factor
        Coe = np.matmul(U_list[0].T,U_list[0])
        Coe = np.linalg.inv(Coe)
        Coe = np.matmul(Coe,U_list[0].T)
        Coe = np.matmul(Coe,A)
        U_list[1] = Coe.T
        
        # update the left factor
        Coe = np.matmul(U_list[1].T,U_list[1])
        Coe = np.linalg.inv(Coe)
        Coe = np.matmul(U_list[1],Coe)
        Coe = np.matmul(A,Coe)
        U_list[0] = Coe
        
        B = np.matmul(U_list[0],U_list[1].T)
        res_new = np.linalg.norm(A - B)/normA
        res_change = np.abs(res_new - res_old)
        res_old = res_new
        num_iter += 1
        
#         print(f"{num_iter}th-iteration: residual is {res_old}, error change is {res_change}\n")
    
    Q,R = np.linalg.qr(U_list[0])
    U_list[0] = Q;
    info = {"factors": U_list, "residual": res_new, "iter": num_iter}
    return info

# unit test
# m,n,r = 10,5,1
# A = np.random.random([m,n])
# # u = np.random.random([m,r])
# # v = np.random.random([n,r])
# # U_init = [u,v]
# info = als2_1(A,r)

In [6]:
# used to calculate the rank-r approximation of a matrix by alternating least squares method with fixed maximum number of iterations
import numpy as np

def als2_2(A,r,init="random",maxiter=2):
    # A: the input matrix
    # r: truncation
    # init: initial factors
    # maxiter: the maximum number of iterations
    
    sizeA = A.shape
    num_iter = 0
    
    # initial factor
    if isinstance(init,np.ndarray):
        U_list = []
        U_list.append(init)
        U_list.append(np.random.random([sizeA[1],r]))
    elif init == "random":
        U_list = []
        U_list.append(np.random.random([sizeA[0],r]))
        U_list.append(np.random.random([sizeA[1],r]))
    
    for i in range(maxiter):
        # update the right factor
        Coe = np.matmul(U_list[0].T,U_list[0])
        Coe = np.linalg.inv(Coe)
        Coe = np.matmul(Coe,U_list[0].T)
        Coe = np.matmul(Coe,A)
        U_list[1] = Coe.T
        
        # update the left factor
        Coe = np.matmul(U_list[1].T,U_list[1])
        Coe = np.linalg.inv(Coe)
        Coe = np.matmul(U_list[1],Coe)
        Coe = np.matmul(A,Coe)
        U_list[0] = Coe
        
    Q,R = np.linalg.qr(U_list[0])
    U_list[0] = Q
    info = {"factors": U_list}
    return info


In [7]:
# uesd to calculate the low multilinear-rank approximation of a tensor by HOOI-ALS
import numpy as np

def hooi_als(A,rank,init="random",als=1,tol=1.0e-4,maxiter=500):
    # A: the input tensor
    # rank: the truncation
    # init: initial factors
    # als: determine the als scheme
    # maxiter: the maximum number of iterations
    
    N = A.ndim
    sizeA = A.shape
    op_index = np.arange(N)
    normA = np.linalg.norm(A)
    num_iter = 0
    
    # initial factors
    if isinstance(init,list):
        U_list = init
    elif init == "random":
        U_list = []
        for n in range(N):
            U_list.append(np.random.random([sizeA[n],rank[n]]))
            Q,R = np.linalg.qr(U_list[n])
            U_list[n] = Q
    
    res_old = 1.0
    res_change = 1.0
    print("\033[1;35;46mALS-based high-order orthogonal iteration:\n\033[0m")
    
    while (res_change > tol and num_iter < maxiter):
        for n in range(N):
            inter_init = U_list[n]
            op_interindex = np.delete(op_index,n)
            U_list.pop(n)
            B = ttmc(A,U_list,op_interindex,transpose=True)
            mat = tenmat(B,n)
            if als == 1:
                info = als2_1(mat,rank[n],inter_init)
            elif als == 2:
                info = als2_2(mat,rank[n],inter_init)
            U = info["factors"]
            U_list.insert(n,U[0])
                
        G = ttmc(A,U_list,op_index,transpose=True)
        normG = np.linalg.norm(G)
        res_new = np.sqrt(np.abs(1 - np.square(normG)/np.square(normA)))
        res_change = np.abs(res_new - res_old)
        res_old = res_new
        num_iter += 1
        
        print(f"{num_iter}th-iteration: residual is {res_new}, error change is {res_change}")
    
    info = {"core tensor": G, "factors": U_list, "residual": res_new, "iter": num_iter}
    return info

In [8]:
# unit test
from tensorly.decomposition import tucker
import numpy as np
import time

# I,J,K,L,M = 10,10,100,100,100
# A = np.random.random([I,J,K,L,M])
# # A = np.empty([I,J,K,L,M])
# # for i in range(I):
# #     for j in range(J):
# #         for k in range(K):
# #             for l in range(L):
# #                 for m in range(M):
# #                     A[i,j,k,l,m] = np.sin(i+j+k+m+l)
# rank = [2,2,2,2,2]

# I,J,K = 100,1000,500
# A = np.random.random([I,J,K])
# rank = [10,10,50]

I,J,K = 200,2000,500
# A = np.empty([I,J,K])
# for i in range(I):
#     for j in range(J):
#         for k in range(K):
#             A[i,j,k] = np.sin(i+j+k)
# A = np.random.normal(size=(I,J,K))
A = np.random.random([I,J,K])
rank = [20,10,10]

time_start = time.time()
info = hooi_als(A,rank,als=1)
time_end = time.time()
t1 = time_end - time_start

time_start = time.time()
core,factor = tucker(A,rank=rank)
time_end = time.time()
t2 = time_end - time_start
normA = np.linalg.norm(A)
normG = np.linalg.norm(core)
residual = np.sqrt(np.abs(1 - np.square(normG)/np.square(normA)))

print(f"TensorLy: residual is {residual}\n")
print(f"ALS-based HOOI: {t1}s, TensorLy: {t2}s")

ALS-based high-order orthogonal iteration:

1th-iteration: residual is 0.49995919437223635, error change is 0.5000408056277637
2th-iteration: residual is 0.49994634320135894, error change is 1.2851170877403728e-05
TensorLy: residual is 0.4999348400404276

ALS-based HOOI: 5.816516160964966s, TensorLy: 16.68782615661621s
